In [ ]:
import random
import rtsvg
rt  = rtsvg.RACETrack()
def randomCircleConfig(n_circles=15, w=128, h=128, r_min=3.0, r_max=15.0, d_intra_circle_min=3.0):
    attempts, circles = 0, []
    while len(circles) < n_circles:
        r = random.uniform(r_min, r_max)
        x = random.uniform(r+d_intra_circle_min, w-r-d_intra_circle_min)
        y = random.uniform(r+d_intra_circle_min, h-r-d_intra_circle_min)
        overlaps = False
        for c in circles:
            d = rt.segmentLength(((x,y), c))
            if c[2] + r + d_intra_circle_min > d : overlaps = True
        if overlaps == False: circles.append((x,y,r))
        attempts += 1
        if attempts > 1000: raise Exception('Failed to generate circles')
    return circles

In [ ]:

# circles = [(25.0,25.0,20.0),(80.0,80.0,30.0),(20.0,100.0,10.0),(25.0,60.0,5.0),(60.0,30.0,5.0),(48.0,48.0,6.0),(40.0,90.0,6.0)]
circles = randomCircleConfig()
cell_map, tri_list, V = rt.laguerreVoronoi2D(circles)
x_min, y_min, x_max, y_max = 1e9, 1e9, -1e9, -1e9
for _pair_ in V:
    x,y = _pair_
    x_min, x_max = min(x_min, x), max(x_max, x)
    y_min, y_max = min(y_min, y), max(y_max, y)
for i in range(len(circles)):
    x,y,r = circles[i]
    x_min, x_max = min(x_min, x-r), max(x_max, x+r)
    y_min, y_max = min(y_min, y-r), max(y_max, y+r)
print(f'x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}')
_border_ = 10.0
x_min, y_min, x_max, y_max = x_min - _border_, y_min - _border_, x_max + _border_, y_max + _border_
#svg_hdr = f'<svg x="0" y="0" width="768" height="768" viewbox="{x_min} {y_min} {x_max-x_min} {y_max-y_min}"><rect x="{x_min}" y="{y_min}" width="{x_max-x_min}" height="{y_max-y_min}" fill="white" />'
svg_hdr = f'<svg x="0" y="0" width="768" height="768" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.6"/>')
# rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])
svg_v, svg_fmto = [], []
for _pair_ in V:
    x,y = _pair_
    svg_v.append(f'<circle cx="{x}" cy="{y}" r="0.5" fill="#000000" stroke="none" />')

_light_gray_ = '#d0d0d0'
for _tri_ in tri_list:
    line_0 = f'<line x1="{circles[_tri_[0]][0]}" y1="{circles[_tri_[0]][1]}" x2="{circles[_tri_[1]][0]}" y2="{circles[_tri_[1]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    line_1 = f'<line x1="{circles[_tri_[1]][0]}" y1="{circles[_tri_[1]][1]}" x2="{circles[_tri_[2]][0]}" y2="{circles[_tri_[2]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    line_2 = f'<line x1="{circles[_tri_[2]][0]}" y1="{circles[_tri_[2]][1]}" x2="{circles[_tri_[0]][0]}" y2="{circles[_tri_[0]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    svg_v.append(line_0), svg_v.append(line_1), svg_v.append(line_2)
    svg_fmto.append(line_0), svg_fmto.append(line_1), svg_fmto.append(line_2)

for i in range(len(cell_map)):
    for j in range(len(cell_map[i])):
        fm_to = cell_map[i][j][0]
        if fm_to[0] == -1 or fm_to[1] == -1:
            if   fm_to[0] == -1:
                _to_ = fm_to[1]
                xy   = V[_to_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            elif fm_to[1] == -1:
                _fm_ = fm_to[0]
                xy   = V[_fm_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            if l is None: l = 100.0
            svg_fmto.append(f'<line x1="{xy[0]}" y1="{xy[1]}" x2="{xy[0]+uv[0]*l}" y2="{xy[1]+uv[1]*l}" stroke="red" stroke-width="0.4" />')    
        else:
            _fm_, _to_ = fm_to[0], fm_to[1]
            svg_fmto.append(f'<line x1="{V[_fm_][0]}" y1="{V[_fm_][1]}" x2="{V[_to_][0]}" y2="{V[_to_][1]}" stroke="black" stroke-width="0.2" />')
for xy in V:
    svg_fmto.append(f'<circle cx="{xy[0]}" cy="{xy[1]}" r="{0.5+random.random()}" fill="none" stroke="#0000ff" stroke-width="0.1" />')
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_v) + svg_ftr,
         svg_hdr + ''.join(svg_circles) + ''.join(svg_fmto) + svg_ftr], spacer=10)